In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = 'lovecraft.txt'# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

129 unique characters


In [3]:
vocab_size = len(vocab) # Length of the vocabulary in chars
embedding_dim = 256 # The embedding dimension
rnn_units = 1024 # Number of RNN units

In [4]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [5]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'model/'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [6]:
tf.train.latest_checkpoint(checkpoint_dir)

'model/ckpt_20'

In [7]:
lovecraft_predict = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

lovecraft_predict.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

lovecraft_predict.build(tf.TensorShape([1, None]))


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.embeddings.Embedding object at 0x7f878ff05610> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f878ff05690>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.GRU object at 0x7f8793ae7910> and <tensorflow.python.keras.layers.embeddings.Embedding object at 0x7f878ff05610>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7f87e8529050> and <tensorflow.python.keras.layers.recurrent_v2.GRU object at 0x7f8793ae7910>).


In [8]:
lovecraft_predict.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 256)            33024     
_________________________________________________________________
gru (GRU)                    (1, None, 1024)           3938304   
_________________________________________________________________
dense (Dense)                (1, None, 129)            132225    
Total params: 4,103,553
Trainable params: 4,103,553
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
def generate_text(model, start_string,temp):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = temp

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
        send_text = ''.join(text_generated)
        #send_text = send_text.split('\n')[0]

    #return (start_string + ''.join(text_generated))
    return send_text#(start_string + ''.join(text_generated))

In [11]:
user_input = 'She crept slowly through the ferns, careful to be silent in the night.'#input('')
print(generate_text(lovecraft_predict, start_string=user_input+u'\n',temp=1.0))


“There were two or through, pured with disappointment to work; then and the interment in the strang, Lady and decked together a-tempeth Capt. 
James, I will nor beyond how he had oddly snapped it. 
And I knew that the world had happened to lip. 
It ran as faded past bearing the time sooth thouse no longer heers that dack—his body jeck, who was the death of the Most Rice and Sydney and his intangible Juanna for Zumbardn for them. 
I have said that some supernatural altering had been so far off. 
Three coffin-grown civilisations sent from the earth and forced the loveliness below, and laden were no less frequently enough to hurry across the dissection built in both difficulty; and in bringing Barzai the voices of Yoth had thrown them with the air and will system fe that this side of the house he had made signs of apprehension. 
That people it was darker and pool which, though his buildings, from the ultimately to be revoked the facts. 
As we replied, it was thou always, unconscious cond

In [12]:
user_input = 'She crept slowly through the ferns, careful to be silent in the night.'#input('')
print(generate_text(lovecraft_predict, start_string=user_input+u'\n',temp=0.87))


“He was rich and curved steeples with the dead world of the most ancient amounts of the ancient world, I studied the thin hope that the bargain was a permanent form of fright and disconcerting him, and he drawn in a former distance seeming to be done about the deserted city of his shini, vocal orial processions converse some pathological traveller’s ect. 
There were more terrible odours, for they were not the tripods to the opposite fish-priests. 
Mr. Bredence of this fought I could not be surprised to hold forth the noise was a mafter and leand the other twilight! 
Even events I regretted—handed down the sky like the damnable Elmed Alber 1928, and that trees with evil shadows resounted to find the fullest expanse of the edifices so small and muffled seas. 
One morning crystal was still lorded in his chaot and its mate. 
“Don’t see it, everyone seemed to be quick.” And in the morning I was considerable exterior, and helped him he seemed at times in the tomb, and a fantastic not very s

In [15]:
user_input = 'She crept slowly through the ferns, careful to be silent in the night.'#input('')
print(generate_text(lovecraft_predict, start_string=user_input+u'',temp=0.25))

 
I am a desire to explain that the thing was made up the strange things of men and descend from the stars and glimmered or the last of the strange days of his sister and louder. 
The strange markings were still there, and a sound will not do to breathe the desert and the stones with the deserted streets and colonial type, while the people had so far as I could not be sure. 
It was the first time since the passage was the first time since the poor devil were all the strange days with the great black stone steps to the door and the strange myths of the strange things that had been a man of great silver key which had been a deep bank wall of the deserted city of man to see the strange things about the deserted horizon as a madman’s raving at the same time been the sinister places I beheld the common cities of the hills, and on the other hand was a deep and sensible and unhallowed cliff, and the strange things were the land of the deserted mansion which the doctor was discovered. 
The con

In [14]:
user_input = 'She crept slowly through the ferns, careful to be silent in the night.'#input('')
print(generate_text(lovecraft_predict, start_string=user_input+u'\n',temp=0.01))


“The party was something wholly clear to me with the strange things and the strange days of many strange designs and of the strange things which have driven me to the southwest and the strange days of my father’s farm the sea and the stars while he saw the substance of the strange beings who had been a strange and absorb which formed the strange cities and into the stars and the strange days of my father’s farm the sea and the stars while he saw the substance of the strange beings who had been a strange and absorbing door and send him about the deserted mansion and shapeless and fear-broken panes of the sea. 
There was a sense of familiarity with the death of his house and grown up a prodigious vaulted chamber and shape and strove to recall the strange characters who did not seem to be a devil and despair that the past was the first time since the poor devil were freely present. 
Some of the first hideous masonry was closed the door and the strange days of my father, who had been a st

In [33]:
## try saving and loading the model

In [23]:
lovecraft_predict.save('model/lovecraft_predict.h5') 

In [30]:
from tensorflow import keras
reconstructed_model = tf.keras.models.load_model('model/lovecraft_predict.h5')

In [32]:
user_input = 'She crept slowly through the ferns, careful to be silent in the night.'#input('')
print(generate_text(reconstructed_model, start_string=user_input+u'\n',temp=1.0))

ght about 500 spades who prese a more induced blood in this beautiful host avoid. 
Or cryh with that lurked delight—the abysses; and disfurbed needlessly through the dreaded door of strange dreams. 
Mr. Phillips, through the houses we had hoped for the day before “15G of The Landwas adverse yet even far ahead. 
After an human person who found me utterly not at the first to cut through the sight, I tried to feel that some supposed to follow what you have done it would disable o’clock. 
There were curious greeter because of their need, was no sound, the dawn was over William, the ebid a lamparies of change. 
Only what I had shewn gnable just where, if you’re telephonesen and the wall were throng forms shewed for blood. 
I shall never cease, but was gradually on their tentacles of whose sea-green shobs overheap faor’s stone’s decorative capt were missing. 
The man at Exer) plunge his llearie, half-wholesome blasphemy upward. 
Could the heavy mist that strange bothered, with their tentacle